# Prototyping a Data Storage Model for ChiantiPy

In [1]:
import os

import numpy as np
import pandas
import matplotlib.pyplot as plt
import h5py
import astropy.units as u
#import ChiantiPy.tools.util as ch_util
#import ChiantiPy.tools.io as ch_io
#import ChiantiPy.core as ch
import fiasco.io
import fiasco

%matplotlib inline

## Data Access Model

CHIANTI has several file formats that it stores for each ion. The most notable are,

* `.elvlc`: energy levels (in cm$^{-1}$) with additional level configuration
* `.wgfa`: wavelengths, oscillator strengths, and Einstein A coefficients for the transitions
* `.scups`: temperatures and effective collision strenghts for each transition. Replaces the old `.splups` files. There are also still `.psplups` files
* Additional files:
  * `.fblvl`: information for calculating free-bound continuum
  * `.cilvl`, `.reclvl`: ionization and recombination rates

Essentially, we want to have a property for each of these files. Each of these properties returns an object with a `__getitem__` method that takes in the keys associated with each of these files. These objects return the relevant data streamed out of the HDF5 file.

Ideally, this file would be built once the first time you download ChiantiPy and then only rebuilt when your installed CHIANTI database gets updated. The filename is then stored at the package level. We'll use our CHIANTI database HDF5 file that we've been using in `synthesizAR`.

In [2]:
test = fiasco.IonBase('fe_9')

In [8]:
test.wgfa

fe/fe_9/wgfa
        
Fields
------
lower level index  -- desc.
lower level label  -- desc.
oscillator strength  -- desc.
radiative decay rate (1 / s) -- desc.
transition wavelength (Angstrom) -- desc.
upper level index  -- desc.
upper level label  -- desc.

Footer
------

Theoretical energies, gf and A-values:

Del Zanna, G., Storey, P. J., Badnell, N. R., Mason, H. E., 2014, A&A, 565, A77
DOI: 10.1051/0004-6361/201323297

Note: the the theoretical energies are the 'best energies', i.e. 
are the empirically-adjusted theoretical energies.

n=6 levels (above level no.865):
O'Dwyer, B., Del Zanna, G., Badnell, N.R., Mason, H. E.,Storey, P.J.,
2012, A&A, 537, A22


Produced for the CHIANTI database v.8 by G. Del Zanna 26 June 2014


 -1

        

## Ion Base Sandbox

In [ ]:
chianti_hdf5_filename = '/Users/willbarnes/.fiasco/chianti_dbase.h5'

In [ ]:
class DataIndexer(object):
    
    def __init__(self,top_level_path):
        self.top_level_path = top_level_path
    
    def __getitem__(self,key):
        with h5py.File(chianti_hdf5_filename,'r') as hf:
            grp = hf[self.top_level_path]
            if key not in grp:
                raise IndexError('{} not a valid dataset for {}'.format(key,self.top_level_path))
            ds = grp[key]
            if ds.attrs['unit'] == 'SKIP':
                data = np.array(ds,dtype=ds.dtype)
            else:
                data = u.Quantity(ds,ds.attrs['unit'],dtype=ds.dtype)
            if '|S' in data.dtype.str:
                data = data.astype(str)
        return data
    
    def __repr__(self):
        with h5py.File(chianti_hdf5_filename,'r') as hf:
            grp = hf[self.top_level_path]
            var_names = [(key,'')
                         if grp[key].attrs['unit']=='SKIP' or grp[key].attrs['unit']==''
                         else (key,'({})'.format(grp[key].attrs['unit'])) 
                         for key in grp]
            footer = grp.attrs['footer']
            
        name_strs = '\n'.join(['{} {} -- desc.'.format(v[0],v[1]) for v in var_names])
        return '''{top_level_path}
        
Fields
------
{vars_and_units}

Footer
------
{footer}
        '''.format(top_level_path=self.top_level_path,vars_and_units=name_strs,footer=footer)
    

In [ ]:
def all_subclasses(cls):
    return cls.__subclasses__() + [g for s in cls.__subclasses__() for g in all_subclasses(s)]

In [ ]:
all_ext = [cls.filetype for cls in all_subclasses(fiasco.io.GenericParser) 
           if hasattr(cls,'filetype') and cls.filetype not in ['abund','ip']]

In [ ]:
class IonBase(object):
    
    def __init__(self,ion_name):
        self.ion_name = ion_name
        self.element = ion_name.split('_')[0]
        #self.Z = ch_util.el2z(self.element)
        self.stage = ion_name.split('_')[-1]
        
    @property
    def abundance(self):
        return DataIndexer('/'.join([self.element,'abundance']))
    
    @property
    def ionization_potential(self):
        return DataIndexer('/'.join([self.element,'ionization_potential']))
        
def add_property(cls,filetype):
    def property_template(self):
        return DataIndexer('/'.join([self.element,self.ion_name,filetype]))
    property_template.__doc__ = 'Data in {} type file'.format(filetype)
    property_template.__name__ = filetype
    setattr(cls,property_template.__name__,property(property_template))
    
for filetype in all_ext:
    add_property(IonBase,filetype)
    

This is specifically for an ion. We could also implement an even more generic class for the other non-ion-specific datasets, e.g. abundance, ionization potential, miscellaneous continuum data. 

Alternatively, when the CHIANTI HDF5 database is created, these could just be broken up by ion appropriately. This would work except for the continuum data which is maybe a special case anyway. 

Basically, we just want to avoid having to index things over and over again. Better to just refer to it by the ion name.

Since this kind of data is used in quite a few places, we could provide it as a generic object. This also makes the CHIANTI data easily accessible without the baggage of the ion object if users want to extend it in anyway.

## Dynamic Method Creation

In [ ]:
class FooBar(object):
    def __init__(self,a):
        self.a = a

In [ ]:
def add_method(cls,i):
    def method_template(self):
        return 'Method {} with a = {}'.format(i,self.a)
    method_template.__doc__ = 'docstring for method {}'.format(i)
    method_template.__name__ = 'method{}'.format(i)
    setattr(cls,method_template.__name__,property(method_template))

In [ ]:
for i in range(10):
    add_method(FooBar,i)

In [ ]:
f = FooBar('HelloWorld')

In [ ]:
print(f.method0)

In [ ]:
f.method9

In [ ]:
class Foo(object):
    a = 1
    b = 2

In [ ]:
Foo.b

In [ ]:
hasattr(Foo,'c')